In [1]:
import sys

sys.path.append('/home/jovyan/work')
%load_ext autoreload
%autoreload 2

In [3]:
from helpers.logger import Logger
from helpers.load import load_interventions, load_topics, save_to_pickle
from helpers.vars import codes, helper_langs, interventions_helper
from helpers.plot import set_size, plot_dates, plot_interventions, plot_before_after_diff_in_diffs
from helpers.calc import build_regression, generate_newcomer_ids_for_df, generate_newcomer_ids_for_df_parallel, count_newcomers_per_day
from helpers.diffs_n_diffs import get_df_from_statsmodels_result, get_diffs_in_diffs_result, compute_diff_in_diff_pre_intra_post
from helpers.retrieval import retrieve_all_editor_activity_levels_for_all_wikis, EDITOR_ALL_ACTIVITY_LEVELS, retrieve_newly_registered_all_langs, retrieve_all_edit_counts
import pandas as pd
import requests
import numpy as np
import random as rd
import time
import matplotlib.pyplot as plt
from helpers.logger import Logger


In [4]:
DATA_PATH = '../../../data/coronawiki'
interventions = load_interventions(f'{DATA_PATH}/interventions2.csv')
column_names = np.loadtxt(f'{DATA_PATH}/mediawiki_history_columns.txt', dtype=str)

In [ ]:
dtypes = {'wiki_db': str, 'event_entity': str, 'event_type': str, 'event_timestamp': str,
       'event_comment':str, 'event_user_id': "Int64", 'event_user_text_historical':str,
       'event_user_text': str, 'event_user_blocks_historical': str,
       'event_user_blocks':str, 'event_user_groups_historical':str,
       'event_user_groups':str, 'event_user_is_bot_by_historical':str,
       'event_user_is_bot_by': str, 'event_user_is_created_by_self': "boolean",
       'event_user_is_created_by_system': "boolean", 'event_user_is_created_by_peer': "boolean",
       'event_user_is_anonymous': "boolean", 'event_user_registration_timestamp': str,
       'event_user_creation_timestamp': str, 'event_user_first_edit_timestamp': str,
       'event_user_revision_count':  "Int64",
       'event_user_seconds_since_previous_revision':  "Int64", 'page_id':  "Int64",
       'page_title_historical': str, 'page_title': str, 'page_namespace_historical':  "Int64",
       'page_namespace_is_content_historical': "boolean", 'page_namespace':  "Int64",
       'page_namespace_is_content': "boolean", 'page_is_redirect': "boolean", 'page_is_deleted': "boolean",
       'page_creation_timestamp': str, 'page_first_edit_timestamp': str,
       'page_revision_count':  "Int64", 'page_seconds_since_previous_revision':  "Int64",
       'user_id':  "Int64", 'user_text_historical': str, 'user_text': str,
       'user_blocks_historical': str, 'user_blocks': str, 'user_groups_historical': str,
       'user_groups': str, 'user_is_bot_by_historical': str, 'user_is_bot_by': str,
       'user_is_created_by_self': "boolean", 'user_is_created_by_system': "boolean",
       'user_is_created_by_peer': "boolean", 'user_is_anonymous': "boolean",
       'user_registration_timestamp': str, 'user_creation_timestamp': str,
       'user_first_edit_timestamp': str, 'revision_id':  "Int64", 'revision_parent_id':  "Int64",
       'revision_minor_edit': "boolean", 'revision_deleted_parts': str,
       'revision_deleted_parts_are_suppressed': "boolean", 'revision_text_bytes':  "Int64",
       'revision_text_bytes_diff':  "Int64", 'revision_text_sha1': str,
       'revision_content_model': str, 'revision_content_format': str,
       'revision_is_deleted_by_page_deletion': "boolean",
       'revision_deleted_by_page_deletion_timestamp': str,
       'revision_is_identity_reverted': "boolean",
       'revision_first_identity_reverting_revision_id':  "Int64",
       'revision_seconds_to_identity_revert':  "Int64",
       'revision_is_identity_revert': "boolean",
       'revision_is_from_before_page_creation': "boolean", 'revision_tags': str}

### Load general Data

In [5]:
def process_lang_history(lang, column_names, dtypes, path=DATA_PATH+'/mw-history/', ending='tsv.bz2', years=[2018, 2019, 2020]):
    df_lang = pd.DataFrame()
    for year in years:
        start = time.time()
        if lang == 'en':
            start = time.time()
            try:
                for month in range(1, 13):
                    #if int(year) == 2020 and month > 9:
                    #    break
                    df_lang = pd.concat([df_lang, pd.read_csv(f'{path}{lang}-{year}-{month:02d}.{ending}', sep='\t', names=list(column_names), dtype=dtypes, warn_bad_lines=True, error_bad_lines=False)])
                    Logger.instance().info(f'Loaded {lang}-{year}-{month:02d} in {time.time() - start}')
            except:
                Logger.instance().info(f'Error when processing {lang}-{year}-{month}')
        else:
            try:
                df_lang = pd.concat([df_lang, pd.read_csv(f'{path}{lang}-{year}.{ending}', sep='\t', names=list(column_names), dtype=dtypes, warn_bad_lines=True, error_bad_lines=False)])
                Logger.instance().info(f'Loaded {lang}-{year} in {time.time() - start}')
            except:
                Logger.instance().info(f'Error when processing {lang}-{year}')
    return df_lang

In [4]:
Logger.instance().info(f'Start')

12-07 12:30 : INFO : Created new singleton instance
12-07 12:30 : INFO : Start


New logging instance for ../../logging//coronawiki.log


In [7]:
df_dict = {}

### User Kind, Datetime, Title normalization

In [6]:
import swifter
import pendulum
from mw.lib import title as mw_t
import time
import traceback

In [18]:
path_lang_codes = f'{DATA_PATH}/pipeline/lang_code.csv'
save_path = f'{DATA_PATH}/pipeline/results'

In [10]:
tz_ = dict(pd.read_csv(path_lang_codes).set_index("code").timezone)
tz = {code: pendulum.timezone(t) for code, t in tz_.items()}

In [19]:
save_path


'../../../data/coronawiki/pipeline/results'

In [15]:
dict_edits_byid['fr']

NameError: name 'dict_edits_byid' is not defined

In [ ]:
for code in codes:
    df_dict[code] = process_lang_history(code, column_names, dtypes=str)

for code in codes:
    start = time.time()
    try:
        # === user kind merged to hear
        df_dict[code]['user_kind'] = df_dict[code].apply(lambda row: 'anonymous' if pd.isna(row.event_user_id) else 'bot' if not pd.isna(row.event_user_is_bot_by) else 'account', axis=1)
        Logger.instance('pipeline').info(f'User-Kind assignment for {code} in {time.time() - start}')
        
        # === Convert timestamp
        df_dict[code]['event_timestamp_t'] = df_dict[code].event_timestamp.swifter.apply(pd.to_datetime) #pd.to_datetime(df_dict[code].event_timestamp)
        # Set UTC date
        df_dict[code]["event_timestamp_t"] = df_dict[code].event_timestamp_t.dt.tz_localize("UTC", ambiguous='NaT', nonexistent='NaT')
        df_dict[code]["date_utc"] = df_dict[code].event_timestamp_t.dt.strftime("%Y%m%d").astype(int)
        # LOCALIZE date
        df_dict[code]["event_timestamp_t"] = df_dict[code].event_timestamp_t.dt.tz_convert(tz_[code])
        df_dict[code]["date"] = df_dict[code].event_timestamp_t.dt.strftime("%Y%m%d").astype(int)
        Logger.instance('pipeline').info(f'Finished date assignment for {code} in {time.time() - start}')
        
        # === Normalize date
        df_dict[code]['page_title_norm'] = df_dict[code].page_title.swifter.apply(lambda title: mw_t.normalize(str(title)) if not pd.isna(title) else np.nan)
        df_dict[code]['page_title_historical_norm'] = df_dict[code].page_title_historical.swifter.apply(lambda title: mw_t.normalize(str(title)) if not pd.isna(title) else np.nan)
        Logger.instance('pipeline').info(f'Finished name normalization for {code} in {time.time() - start}')
    except Exception as e:
        traceback.print_exc()
        Logger.instance('pipeline').info(f'Error for {code}: {str(e)}')

In [ ]:
# write to dumps
for code in codes:
    try:
        start = time.time()
        df_dict[code].to_csv(f'{DATA_PATH}/pipeline/enriched/{code}_mwh_processed.tsv.gz', header=True, index=False, sep="\t", compression='gzip')
        Logger.instance('pipeline').info(f'Dumping {code} done in {time.time() - start}')
    except Exception as e:
        traceback.print_exc()
        Logger.instance('pipeline').info(f'Error when saving {code}: {str(e)}')

12-09 12:05 : INFO : Dumping de done in 2596.5580372810364
12-09 12:55 : INFO : Dumping fr done in 3010.0727665424347
12-09 13:29 : INFO : Dumping it done in 2014.8424730300903
12-09 13:41 : INFO : Dumping sr done in 709.250214099884
12-09 13:45 : INFO : Dumping no done in 248.3036527633667
12-09 13:56 : INFO : Dumping ko done in 690.7447185516357
12-09 13:58 : INFO : Dumping da done in 104.91997742652893
12-09 14:07 : INFO : Dumping sv done in 534.8345317840576
12-09 14:26 : INFO : Dumping ja done in 1133.3621723651886
12-09 14:36 : INFO : Dumping nl done in 580.7617044448853
12-09 14:39 : INFO : Dumping fi done in 197.9853708744049


### Group By Date and User_Kind to get newcomers

In [116]:
from helpers.load import save_to_pickle

In [187]:
dict_creation = {}
for code in codes:
    start = time.time()
    try:
        df_code = df_dict[code]
        # define masks
        create_event_mask = (df_code.event_entity=='user') & (df_code.event_type == 'create')
        create_revision_mask = (df_code.event_entity=='revision') & (df_code.event_type == 'create')
        no_bot_mask = (df_code['event_user_is_bot_by'].isna() | df_code['event_user_is_bot_by_historical'].isna())
        self_creation_mask = (df_code['event_user_is_created_by_self'] == 'true')
        no_anon_mask = (df_code['event_user_is_anonymous'] != 'true')

        # === get users by registration
        group_creation = df_code[create_event_mask & no_anon_mask & no_bot_mask & self_creation_mask].groupby(['date'])['event_user_id'].size()

        # === get user by nth edit
        # n=1
        edit_count_mask = df_code.event_user_revision_count == '1'
        group_edit1 = df_code[create_revision_mask & no_anon_mask & no_bot_mask & edit_count_mask].groupby(['date'])['event_user_id'].size().rename('edit_1')
        # n=5
        edit_count_mask = df_code.event_user_revision_count == '5'
        group_edit5 = df_code[create_revision_mask & no_anon_mask & no_bot_mask & edit_count_mask].groupby(['date'])['event_user_id'].size().rename('edit_5')
        dict_creation[code] = pd.concat([group_creation, group_edit1, group_edit5], axis=1).fillna(0)
        Logger.instance('pipeline').info(f'User creation computation for {code} done in {time.time() - start}')
    except Exception as e:
        traceback.print_exc()
        Logger.instance('pipeline').info(f'Error for {code}: {str(e)}')
        
save_to_pickle(f'{save_path}/dict_newcomers_selfcreated.pkl', dict_creation)
Logger.instance('pipeline').info(f'Finished newcomers')

10-05 07:32 : INFO : User creation computation for de done in 50.3608341217041
INFO:pipeline:User creation computation for de done in 50.3608341217041
10-05 07:33 : INFO : User creation computation for fr done in 56.80771470069885
INFO:pipeline:User creation computation for fr done in 56.80771470069885
10-05 07:34 : INFO : User creation computation for it done in 32.6069860458374
INFO:pipeline:User creation computation for it done in 32.6069860458374
10-05 07:34 : INFO : User creation computation for sr done in 11.401366949081421
INFO:pipeline:User creation computation for sr done in 11.401366949081421
10-05 07:34 : INFO : User creation computation for no done in 4.449726104736328
INFO:pipeline:User creation computation for no done in 4.449726104736328
10-05 07:34 : INFO : User creation computation for ko done in 12.441081523895264
INFO:pipeline:User creation computation for ko done in 12.441081523895264
10-05 07:34 : INFO : User creation computation for da done in 1.7098534107208252
I

### Group By Date and Page and user_kind to get edits per day (see above)

In [165]:
dict_edits_byid = {}
dict_edits_bytitle = {}

In [176]:
# Group By Date and Page and user_kind to get edits per day
dict_edits_byid = {}
dict_edits_bytitle = {}
for code in codes:
    try:
        # include redirects?
        start = time.time()
        df_code = df_dict[code]
        create_revision_mask = (df_code.event_entity=='revision') & (df_code.event_type == 'create')
        ns_mask = df_code.page_namespace == '0'

        # group by date, page_id, user_kind
        df_code.revision_text_bytes = pd.to_numeric(df_code['revision_text_bytes'], errors='coerce').fillna(0)
        df_code_masked = df_code[create_revision_mask & ns_mask]
        
        dict_edits_byid[code] = df_code_masked.groupby(['date', 'page_id', 'user_kind']).agg(
            {'event_user_id': 'size', 'revision_text_bytes': 'sum', 'page_title': 'last', 
             'page_title_norm': 'last', 'page_title_historical_norm': 'last'})
        dict_edits_bytitle[code] = df_code_masked.groupby(['date', 'page_title_norm', 'user_kind']).agg(
            {'event_user_id': 'size', 'revision_text_bytes': 'sum', 'page_id': lambda x: set(x), 'page_title': lambda x: set(x),  'page_title_historical_norm': lambda x: set(x)})

        Logger.instance('pipeline').info(f'Grouped by user/user_kind/pageid for {code} done in {time.time() - start}')
    except Exception as e:
        traceback.print_exc()
        Logger.instance('pipeline').info(f'Error for {code}: {str(e)}')  
    
save_to_pickle(f'{save_path}/dict_edits_byid.pkl', dict_edits_byid)
save_to_pickle(f'{save_path}/dict_edits_bytitle.pkl', dict_edits_bytitle)
Logger.instance('pipeline').info(f'Finished edits')

10-04 17:55 : INFO : Grouped by user/user_kind/pageid for nl done in 38.04203796386719
INFO:pipeline:Grouped by user/user_kind/pageid for nl done in 38.04203796386719
10-04 18:12 : INFO : Grouped by user/user_kind/pageid for en done in 1010.5985779762268
INFO:pipeline:Grouped by user/user_kind/pageid for en done in 1010.5985779762268
10-04 18:19 : INFO : Finished edits
INFO:pipeline:Finished edits


### Group By Date and user_kind to get identity reverts per day (see above)

In [261]:
dict_reverts = {}
for code in codes:
    try:
        start = time.time()
        df_code = df_dict[code]
        create_revision_mask = (df_code.event_entity=='revision') & (df_code.event_type == 'create')
        ns_mask = df_code.page_namespace == '0'

        # get reverts per day as well as reverted
        df_reverted = df_code[create_revision_mask & ns_mask & (df_code.revision_is_identity_reverted == 'true')].groupby(['date', 'user_kind'])['revision_is_identity_reverted'].size()
        df_reverts = df_code[create_revision_mask & ns_mask & (df_code.revision_is_identity_revert == 'true')].groupby(['date', 'user_kind'])['revision_is_identity_revert'].size()
        
        # reindex so all dates are filled
        df_reverted = df_reverted.reindex(
            pd.MultiIndex.from_product([df_code.date.unique(), df_reverted.index.levels[1]], names=['date', 'user_kind']), fill_value=0)
        df_reverts = df_reverts.reindex(
            pd.MultiIndex.from_product([df_code.date.unique(), df_reverts.index.levels[1]], names=['date', 'user_kind']), fill_value=0)
    
        dict_reverts[code] = pd.concat([df_reverted, df_reverts], axis=1).fillna(0)
        Logger.instance('pipeline').info(f'Computed reverts by {code} done in {time.time() - start}')
    except Exception as e:
        traceback.print_exc()
        Logger.instance('pipeline').info(f'Error for {code}: {str(e)}')  
    
save_to_pickle(f'{save_path}/dict_reverts.pkl', dict_reverts)
Logger.instance('pipeline').info(f'Finished identity reverts')

10-05 15:20 : INFO : Computed reverts by de done in 44.71893262863159
INFO:pipeline:Computed reverts by de done in 44.71893262863159
10-05 15:21 : INFO : Computed reverts by fr done in 57.87290406227112
INFO:pipeline:Computed reverts by fr done in 57.87290406227112
10-05 15:21 : INFO : Computed reverts by it done in 32.34038710594177
INFO:pipeline:Computed reverts by it done in 32.34038710594177
10-05 15:21 : INFO : Computed reverts by sr done in 7.730040788650513
INFO:pipeline:Computed reverts by sr done in 7.730040788650513
10-05 15:21 : INFO : Computed reverts by no done in 3.897212028503418
INFO:pipeline:Computed reverts by no done in 3.897212028503418
10-05 15:22 : INFO : Computed reverts by ko done in 11.912227630615234
INFO:pipeline:Computed reverts by ko done in 11.912227630615234
10-05 15:22 : INFO : Computed reverts by da done in 1.4106924533843994
INFO:pipeline:Computed reverts by da done in 1.4106924533843994
10-05 15:22 : INFO : Computed reverts by sv done in 10.2345514297

### Combine with topics

In [18]:
df_topics, topics = load_topics(f'{DATA_PATH}/pipeline/topicsGen/topics_linked_ratio.f')

In [ ]:
# we dropped some articles here. do we want to get them back? -> basically: group by name, or normalized name?

In [19]:
df_topics

,index,Geography.Regions.Africa.Africa*,Geography.Regions.Asia.East Asia,Geography.Regions.Africa.Southern Africa,Culture.Philosophy and religion,Geography.Regions.Europe.Western Europe,Geography.Regions.Asia.West Asia,Culture.Media.Music,Culture.Linguistics,Geography.Regions.Asia.North Asia,...,Geography.Regions.Americas.Central America,Culture.Internet culture,STEM.Mathematics,Culture.Media.Films,Geography.Geographical,STEM.Computing,Geography.Regions.Africa.Eastern Africa,Culture.Visual arts.Comics and Anime,qid,edit_ratio
0,Emperor_Kōtoku,0.0,0.333333,0.0,0.0,0.00,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Q317980,0.333333
1,Chemical_affinity,0.0,0.000000,0.0,0.0,0.00,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Q382783,0.500000
2,Farmer_Giles_of_Ham,0.0,0.000000,0.0,0.0,0.00,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Q471398,0.333333
3,Transpositional_pun,0.0,0.000000,0.0,0.0,0.00,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Q7835334,0.500000
4,Sampler_(needlework),0.0,0.000000,0.0,0.0,0.00,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Q1513987,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1618404,Enno_Stephan,0.0,0.000000,0.0,0.0,0.25,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Q83795836,0.250000
1618405,"Seigan-ji_(Atsuta-ku,_Nagoya)",0.0,0.500000,0.0,0.0,0.00,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Q17220718,0.500000
1618406,Phill_Grimshaw,0.0,0.000000,0.0,0.0,0.00,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Q81807039,0.333333
1618407,Csepel_D-344,0.0,0.000000,0.0,0.0,0.00,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Q9196926,0.250000


In [295]:
ra = pd.read_csv(f'{DATA_PATH}/pipeline/pageid_enwiki.csv')

## Combine edit dictionary with covid info

In [10]:
from mw.lib import title as mw_t
path_covid = f'{DATA_PATH}/pipeline/topicsGen/covid_linked.f'

df_covid = pd.read_feather(path_covid)
df_covid['covid'] = True
df_covid['index'] = df_covid['index'].apply(lambda t: mw_t.normalize(str(t)))

In [13]:
dict_edits = load_from_pickle(f'{PIP_PATH}/results/dict_edits_bytitle.pkl')

In [19]:
df_edits_covid = {}
for code, df_code in dict_edits.items():
    df_edits_covid[code] = df_code.reset_index().merge(df_covid[df_covid.site == f'{code}wiki'], left_on=['page_title_norm'], right_on=['index'], how='left').fillna({'covid': False}).drop(['index', 'site', 'qid'], axis=1)

In [20]:
save_to_pickle(f'{PIP_PATH}/results/dict_edits_bytitle_covid.pkl', df_edits_covid)

In [21]:
df_edits_covid['de']

,date,page_title_norm,user_kind,event_user_id,revision_text_bytes,page_id,page_title,page_title_historical_norm,covid
0,20180101,'''Vorwärts'''_ist_der_Name_u.a._von_diversen_...,account,1,563.0,{10171770},{'''Vorwärts'''_ist_der_Name_u.a._von_diversen...,{'''Vorwärts'''_ist_der_Name_u.a._von_diversen...,False
1,20180101,"((Benutzer_Diskussion:Geist,_der_stets_vernein...",account,2,5297.0,{10171340},"{((Benutzer_Diskussion:Geist,_der_stets_vernei...","{((Benutzer_Diskussion:Geist,_der_stets_vernei...",False
2,20180101,(1)_Ceres,account,1,27279.0,{30371},{(1)_Ceres},{(1)_Ceres},False
3,20180101,(1)_Ceres,anonymous,1,27699.0,{30371},{(1)_Ceres},{(1)_Ceres},False
4,20180101,(1957)_Angara,account,1,3756.0,{7760419},{(1957)_Angara},{(1957)_Angara},False
...,...,...,...,...,...,...,...,...,...
14243014,20201201,Ķekava,account,1,2134.0,{118473},{Ķekava},{Ķekava},False
14243015,20201201,Ļaudona,account,2,5582.0,{3246165},{Ļaudona},{Ļaudona},False
14243016,20201201,Święta_Lipka,account,1,17675.0,{1126168},{Święta_Lipka},{Święta_Lipka},False
14243017,20201201,Żaklin_Nastić,bot,1,4021.0,{10062062},{Żaklin_Nastić},{Żaklin_Nastić},False


### Aggregate

In [1]:
from mw.lib import title as mw_t
from scipy.stats import entropy
import swifter
from helpers.load import load_from_pickle, load_topics, load_covid_related
import traceback
from scipy.stats import entropy
from helpers.load import load_covid_related
from helpers.pipeline import load_preprocess_results, aggregate_preprocess_results, load_full_topics

ModuleNotFoundError: No module named 'helpers'

In [2]:
PIP_PATH = '../../../data/coronawiki/pipeline'
RES_PATH= f'../../../data/coronawiki/pipeline/results'

In [17]:
path_interlanguage_links = f'{DATA_PATH}/pipeline/interlanguagelinks.parquet'
path_lang_codes = f'{DATA_PATH}/pipeline/lang_code.csv'
path_covid = f'{DATA_PATH}/pipeline/topicsGen/covid_linked.f'
path_topics = f'{DATA_PATH}/pipeline/topicsGen/topics_linked_ratio.f'

groupby_author_path = f'{RES_PATH}/dict_newcomers_selfcreated.pkl'
groupby_reverts_path = f'{RES_PATH}/dict_reverts.pkl'
path_agg_out = f'{RES_PATH}/aggregated_new.tsv.gz'
date_range = [int(f'{date.year:02d}{date.month:02d}{date.day:02d}') for date in pd.date_range(pd.to_datetime('2018-01-01'), pd.to_datetime('2020-09-01'), freq='d')]

In [ ]:
codes, df_topics, topics, df_il, dict_date, dict_newcomers, dict_reverts = load_preprocess_results(
    PIP_PATH, 'interlanguagelinks.parquet', 'lang_code.csv', 'topicsGen/topics_linked_ratio.f',
    'results/dict_edits_bytitle_covid.pkl', 'results/dict_newcomers_selfcreated.pkl', 'results/dict_reverts.pkl')

final_aggs = aggregate_preprocess_results(list(dict_date.keys()), df_topics, topics, df_il, dict_date, dict_newcomers, dict_reverts)
#final_aggs.to_csv(path_agg_out, index=False, sep="\t", compression="gzip")
final_aggs.to_csv('../../../data/coronawiki/pipeline/results/aggregated_topics.tsv.gz', index=False, sep="\t", compression="gzip")

In [116]:
entropydf = df_linked.groupby(["date", "user_kind"])["count"].apply(entropy).reset_index()
entropydf.rename({"count": "entropy"}, inplace=True, axis=1)
final = final.merge(entropydf, left_on=["date", "user_kind"], right_on=["date", "user_kind"], how="left")

In [23]:
final_aggs[]

,date,covid,user_kind,index,count,rev_len_sum,entropy,count:Culture.Biography.Biography*,rev_len_sum:Culture.Biography.Biography*,count_ratio:Culture.Biography.Biography*,...,count:Unclassified,rev_len_sum:Unclassified,count_ratio:Unclassified,rev_len_sum_ratio:Unclassified,actor_user,edit_1,edit_5,revision_is_identity_reverted,revision_is_identity_revert,code
0,2018-01-01,False,account,69415429,16982,317421983.0,8.978807,4132.0,83900765.0,1116.619968,...,268.0,8523798.0,268.0,8523798.0,232.0,131.0,32.0,400,742,de
1,2018-01-01,False,anonymous,8191385,1979,44773852.0,6.771333,559.0,12193444.0,159.859571,...,35.0,1300488.0,35.0,1300488.0,0.0,0.0,0.0,580,89,de
2,2018-01-01,False,bot,1540257,267,10845764.0,5.463911,20.0,764138.0,6.216667,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12,6,de
3,2018-01-02,False,account,254740364,20220,365770292.0,9.117339,4591.0,102386561.0,1219.702581,...,299.0,9455184.0,299.0,9455184.0,359.0,214.0,52.0,704,1108,de
4,2018-01-02,False,anonymous,37378321,2526,55281252.0,7.338263,680.0,15983991.0,177.513672,...,59.0,2800900.0,59.0,2800900.0,0.0,0.0,0.0,652,128,de
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4794,2020-12-01,False,anonymous,51795874095,1109,37501331.0,6.324511,445.0,14477553.0,129.044841,...,42.0,1369445.0,42.0,1369445.0,0.0,0.0,0.0,179,54,en
4795,2020-12-01,False,bot,100221780010,1369,34922996.0,7.223744,503.0,11970909.0,144.207937,...,24.0,982753.0,24.0,982753.0,0.0,0.0,0.0,0,35,en
4796,2020-12-01,True,account,146520251,2,217876.0,7.402745,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,95.0,64.0,14.0,69,268,en
4797,2020-12-01,True,anonymous,73260123,4,286493.0,6.324511,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,179,54,en
